<a href="https://colab.research.google.com/github/ArshadP443/Pokemon-Classifier/blob/main/Arshad_Pokemon_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv('pokemon-data.csv', delimiter=';') #Open the file
df["Total"] = df["HP"] + df["Attack"] + df["Defense"] + df["Special Attack"] + df["Special Defense"] + df["Speed"] # Add a new column for total / these columns are used for augmentation
df["Tier"] = df["Tier"].fillna("none") #Prevent future errors with null
pokemon = df.drop("Name", axis = 'columns') # Get rid of names (not useful)
pokemon = pokemon.drop("Next Evolution(s)", axis = 'columns') # Also not very useful
pokemon = pokemon.drop_duplicates() # Remove duplicates
print(len(pokemon["HP"]))
pokemon

In [ ]:
column = list(pokemon.columns.values)
for x in column[3:9]: # Augmentation - Add and subtract 1 to every column used in the augmentation listed above
  for y in range(0, 816):
    row = pokemon.iloc[y]
    row[x] += 1
    row["Total"] += 1
    pokemon = pokemon._append(row, ignore_index = True)
    pokemon = pokemon.drop_duplicates()
    row = pokemon.iloc[y]
    row[x] -= 1
    row["Total"] -= 1
    pokemon = pokemon._append(row, ignore_index = True)
    pokemon = pokemon.drop_duplicates()
pokemon

In [ ]:
pokemon.to_csv("Augmented_Pokemon_Final_.csv") # Save new dataframe to new csv

In [ ]:
from sklearn.preprocessing import OrdinalEncoder # Allows model to analyze strings
Tiers = pokemon["Tier"]
pokemon = pokemon.drop("Tier", axis = "columns")
encode = OrdinalEncoder()
encode.fit(pokemon)
encode.transform(pokemon)

In [ ]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
enc.fit(Tiers)

In [ ]:
Tiers.value_counts()

Tier
LC       3097
PU       2586
none     1129
RU        747
UU        690
NU        660
OU        632
Uber      550
UUBL      249
RUBL      132
NUBL      117
PUBL       92
Limbo      14
AG         13
Name: count, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

# normal process of splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(
    encode.transform(pokemon),
    enc.transform(Tiers),
    #random_state = 42,
    test_size=0.2,
)

print(X_train)
print(Y_train)
print(X_test.shape)
print(Y_test.shape)
#Dan DeGenaro (my instructor) taught me how to write this code

[[ 50.  19.  34. ...  62. 843. 238.]
 [ 43. 522.  62. ...  93. 773. 337.]
 [ 50.  19.  53. ...  52. 788. 306.]
 ...
 [ 48. 152.  46. ... 101. 152. 256.]
 [ 54.  32.  85. ...  21.  42. 237.]
 [173. 126.  39. ...  77. 286. 215.]]
[10 11  5 ...  3  3 13]
(2142, 10)
(2142,)


In [ ]:
from sklearn.preprocessing import StandardScaler

# initialize a scaler, just like a model
scaler = StandardScaler()

# fit the scaler on the training data, and transform it as well while you're at it
X_train = scaler.fit_transform(X_train)

# ONLY transform the test data (NOT fit)
X_test = scaler.transform(X_test)

#Mr. DeGenaro also taught me how to do this as well

In [ ]:
#SVC results:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
model = SVC(
    #class_weight = {0:1, 1: 2, 2:1, 3:3},
    kernel= "rbf",
    C = 3,
    degree = 1,
    gamma = 'scale',
    decision_function_shape = 'ovo',
    probability = True,
)
model.fit(X_train, Y_train)
Prediction = model.predict(X_test)
print(Prediction, Prediction.shape)
print(confusion_matrix(Y_test, Prediction))
print(classification_report(Y_test, Prediction))
print(enc.inverse_transform([0,1,2,3,4,5,6,7,8,9,10,11,12,13]))
#He also helped me with this

[3 6 1 ... 6 5 6] (2142,)
[[  2   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0 611   0   0   0   0   0   0   0   0   0   0   0   8]
 [  0   0   3   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0 114   0   0  30   0   0   0   0   0   0   0]
 [  0   0   0   0  24   0   0   0   0   0   0   0   0   0]
 [  0   0   0   1   0 119   1   0   0   0   6   0   1   0]
 [  0  11   0   0   0   2 513   0   1   0   0   0   0  10]
 [  0   0   0   0   0   0   0  19   0   0   0   0   0   0]
 [  0   0   0   0   0   0  11   0 128   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  22   0   0   0   0]
 [  0   0   0   0   0   0   6   0   0   0 128   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0  48   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0  95   0]
 [  0  14   0   0   0   0   2   0   0   0   0   0   0 212]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       0.96      0.99     

In [ ]:
"""
#SVC Training
from sklearn.model_selection import GridSearchCV
param_grid = {
    "class_weight": [{0:1, 1:2, 2:1, 3:3}],
    "C": [.35, .3],
    "kernel": ['rbf'],
    "degree": [1, 2, 3],
    "gamma": [.1, 'scale', 'auto'],
    "decision_function_shape": ['ovo'],
    "probability": [True],
}
grid_search = GridSearchCV(SVC(),
                           param_grid=param_grid)
grid_search.fit(X_train, Y_train)
print(grid_search.best_estimator_)
"""
# Code taken from https://www.geeksforgeeks.org/hyperparameter-tuning-using-gridsearchcv-and-kerasclassifier/?ref=oin_asr5

In [ ]:
"""
#DecisionTreeClassifier best results:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report
model = DecisionTreeClassifier(
    class_weight = {0:1, 1:2, 2:.8, 3:2},
    max_depth = 200,
    max_features = 10000,
    min_weight_fraction_leaf = .1,
    min_impurity_decrease=0
)
model.fit(X_train, Y_train)
Prediction = model.predict(X_test)
print(Prediction, Prediction.shape)
print(confusion_matrix(Y_test, Prediction))
print(classification_report(Y_test, Prediction))
print(enc.inverse_transform([0,1,2,3]))
"""
# Testing other models

In [ ]:
"""
#DesicionTreeClassifier Training
from sklearn.model_selection import GridSearchCV
param_grid = {
    "criterion": ["gini","entropy","log_loss"],
    'max_depth': [200,250,300],
    "min_weight_fraction_leaf": [0.0, .1, .2],
    "max_features": ['sqrt', 'log2', 100000, 100],
    "min_impurity_decrease": [0, 1, 10, 50, 100],
    "class_weight": ["balanced", {0:1, 1:2, 2:1, 3:2}, {0:1, 1:2, 2:.8, 3:2}, {0:1, 1:2, 2:.9, 3:4}]
}
grid_search = GridSearchCV(DecisionTreeClassifier(),
                           param_grid=param_grid)
grid_search.fit(X_train, Y_train)
print(grid_search.best_estimator_)
"""
# Tested this model in similar ways

In [ ]:
ndf = pd.DataFrame(encode.transform(pokemon))
ndf.style.background_gradient(cmap ='viridis').set_properties(**{'font-size': '20px'})
# Heatmap to see what allowed for strong results (very varied/depends on the pokemon)

In [ ]:
pokemon